In [16]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Font, Alignment, Border, Side

# ===== Excel読み込み =====
# ※ファイル名は適宜環境に合わせて変更してください
df_2022 = pd.read_excel("2022_年間売上表.xlsx", sheet_name="Sheet1")
df_2023 = pd.read_excel("2023_年間売上表.xlsx", sheet_name="Sheet1")

# ===== データ連結 =====
df_all = pd.concat([df_2022, df_2023], ignore_index=True)
df_all.columns = df_all.columns.str.strip()

# ===== 集計 =====
df_summary = (
    df_all
    .groupby(["商品", "売上年"], as_index=False)["金額（千円）"]
    .sum()
)

# 【追加】ナンバリング（No列）の作成
# 先頭（0番目の列）に1から始まる連番を挿入します
df_summary.insert(0, "No", range(1, len(df_summary) + 1))

# ===== Excelへ出力 =====
output_file = "売上集計表.xlsx"
df_summary.to_excel(output_file, index=False)

# ===== Excel 装飾 =====
wb = load_workbook(output_file)
ws = wb.active

gray_fill = PatternFill(start_color="F2F2F2", end_color="F2F2F2", fill_type="solid")
bold_font = Font(bold=True)
center = Alignment(horizontal="center", vertical="center")
right = Alignment(horizontal="right", vertical="center")
left = Alignment(horizontal="left", vertical="center")
thin = Border(left=Side(style="thin"), right=Side(style="thin"),
              top=Side(style="thin"), bottom=Side(style="thin"))

# ヘッダー装飾
for cell in ws[1]:
    cell.fill = gray_fill
    cell.font = bold_font
    cell.alignment = center
    cell.border = thin

# データ部分装飾（列が増えたためインデックスを調整）
for row in ws.iter_rows(min_row=2):
    row[0].alignment = center  # No
    row[1].alignment = left    # 商品
    row[2].alignment = center  # 売上年
    row[3].alignment = right   # 金額
    for cell in row:
        cell.border = thin

# 列幅調整（No列用にA列を追加し、以降をスライド）
ws.column_dimensions["A"].width = 5   # No
ws.column_dimensions["B"].width = 18  # 商品
ws.column_dimensions["C"].width = 10  # 売上年
ws.column_dimensions["D"].width = 12  # 金額

wb.save(output_file)
print("完了: ナンバリング付きで 売上集計表.xlsx を作成しました")

完了: ナンバリング付きで 売上集計表.xlsx を作成しました
